# Control Systems 1, NB09: The Nyquist Condition
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers various transfer function formulations, and how they can be used to graphically determine the magnitude and phase of the transfer function. Furthermore, the effects of poles and zeros on the output response are briefly covered.

Authors:
- Jonas Holinger; jholinger@ethz.ch
- Shubham Gupta; shugupta@ethz.ch

## Learning Objectives

After completing this notebook, you should be able to:


# Setup


## Installing the required packages:

In [ ]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

## Import the packages


The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import sympy as sp
import control as ct
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from scipy.integrate import odeint
from scipy.signal import zpk2tf, residue, invres
from IPython.display import display, clear_output, Math, Latex
from ipywidgets import interactive

# Introduction to Nyquist Stability Criterion
In this section, we'll explore fundamental concepts in control systems that help us determine the stability of a closed-loop system by analyzing its open-loop transfer function.

We'll start by understanding the **Principle of the Argument**, then learn about the **Nyquist Plot**, and finally delve into the **Nyquist Stability Criterion** itself.


## Principle of Variation of the Argument


The **Principle of the Argument** is a concept from complex analysis that relates the number of zeros and poles of a complex function inside a closed contour to the net change in the argument (angle) of the function along that contour.

### Mathematical Explanation

For a complex function \( F(s) \), where \( s \) is a complex variable, the principle states:

\[
N = Z - P
\]

- **\( N \)**: Net number of times the function's plot encircles the origin (counter-clockwise direction counted as positive) as \( s \) traverses the contour.
- **\( Z \)**: Number of zeros of \( F(s) \) inside the contour.
- **\( P \)**: Number of poles of \( F(s) \) inside the contour.

### Intuitive Understanding

Imagine walking along a path in the complex plane, tracing the function \( F(s) \) as \( s \) moves along a closed contour. The principle tells us that the total number of times you wind around the origin (counting direction) equals the number of zeros minus the number of poles inside the path.


## Nyquist Plot


A **Nyquist Plot** is a graphical representation of a complex transfer function \( L(s) \) when \( s = j\omega \), where \( \omega \) ranges from \(-\infty\) to \( \infty \). It maps the frequency response of the open-loop system onto the complex plane.

### How to Construct a Nyquist Plot

1. **Determine \( L(j\omega) \)**: Substitute \( s = j\omega \) into the open-loop transfer function \( L(s) \).
2. **Calculate Magnitude and Phase**: For each frequency \( \omega \), compute the magnitude \( |L(j\omega)| \) and phase angle \( \angle L(j\omega) \).
3. **Plot on Complex Plane**: Plot the complex numbers \( L(j\omega) \) on the real-imaginary plane as \( \omega \) varies from \(-\infty\) to \( \infty \).

### Key Features

- **Symmetry**: For systems with real coefficients, the Nyquist plot is symmetrical about the real axis.
- **Encirclements**: The way the plot encircles the critical point \(-1 + j0\) is crucial for stability analysis.
- **Mapping Frequency**: Low frequencies start near the origin, while high frequencies extend outward.

### Visual Interpretation

The Nyquist plot provides a visual means to assess how the system's gain and phase shift affect stability. By observing how the plot wraps around the critical point, we can infer the system's tendency to oscillate or remain stable.


## Nyquist Stability Criterion

### Purpose

The **Nyquist Stability Criterion** allows us to determine the stability of a closed-loop control system by examining its open-loop transfer function \( L(s) \) without explicitly calculating the closed-loop poles.

### The Criterion Explained

Given:

- **\( P \)**: Number of poles of \( L(s) \) in the right-half of the complex plane (unstable poles).
- **\( N \)**: Net number of clockwise encirclements of the critical point \(-1 + j0\) by the Nyquist plot of \( L(j\omega) \).
- **\( Z \)**: Number of zeros of \( 1 + L(s) \) in the right-half plane (closed-loop poles in the RHP).

The Nyquist Stability Criterion states:

\[
Z = N + P
\]

**Interpretation:**

- If \( Z = 0 \), all closed-loop poles are in the left-half plane (stable system).
- A positive \( Z \) indicates the number of unstable poles in the closed-loop system.

### Applying the Criterion Step-by-Step

1. **Identify \( P \)**: Count the number of RHP poles of the open-loop transfer function \( L(s) \).
2. **Plot Nyquist Diagram**: Create the Nyquist plot for \( L(j\omega) \).
3. **Determine \( N \)**: Count the net number of clockwise encirclements of \(-1 + j0\).
   - Clockwise encirclement: \( N > 0 \).
   - Counter-clockwise encirclement: \( N < 0 \).
4. **Compute \( Z \)**: Use \( Z = N + P \).
5. **Assess Stability**: If \( Z = 0 \), the system is stable.

### Important Considerations

- **Contour Selection**: The Nyquist contour must encircle the entire right-half s-plane.
- **Poles on Imaginary Axis**: If \( L(s) \) has poles on the imaginary axis, modifications to the criterion are needed.
- **Infinite Frequencies**: The plot includes behavior at both \( \omega = 0 \) and \( \omega = \infty \).

### Example Illustration

Suppose \( L(s) \) has one RHP pole (\( P = 1 \)).

- **Case 1**: Nyquist plot encircles \(-1 + j0\) once clockwise (\( N = 1 \)).
  - \( Z = N + P = 1 + 1 = 2 \).
  - The closed-loop system has two RHP poles (\( Z = 2 \)) → **Unstable**.
  
- **Case 2**: Nyquist plot does not encircle \(-1 + j0\) (\( N = 0 \)).
  - \( Z = N + P = 0 + 1 = 1 \).
  - The closed-loop system has one RHP pole (\( Z = 1 \)) → **Unstable**.
  
- **Case 3**: Nyquist plot encircles \(-1 + j0\) once counter-clockwise (\( N = -1 \)).
  - \( Z = N + P = -1 + 1 = 0 \).
  - The closed-loop system has no RHP poles (\( Z = 0 \)) → **Stable**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control

# Define the open-loop transfer function L(s)
# Example: L(s) = 2(s + 5)(s - 0.1) / [s(s + 1)(s + 10)]
numerator = [2, 9.8, -1]  # 2*(s + 5)*(s - 0.1) = 2s^2 + 9.8s - 1
denominator = [1, 11, 10, 0]  # s(s + 1)(s + 10) = s^3 + 11s^2 + 10s

L = control.TransferFunction(numerator, denominator)

# Plot Nyquist diagram
control.nyquist(L, Plot=True)
plt.title('Nyquist Plot of L(s)')
plt.show()

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import control

def parse_coefficients(coeff_str):
    """
    Parse a string of coefficients separated by commas into a list of floats.
    """
    try:
        coeffs = [float(c.strip()) for c in coeff_str.split(',') if c.strip()]
        if not coeffs:
            raise ValueError("Coefficient list cannot be empty.")
        return coeffs
    except ValueError as e:
        raise ValueError(f"Invalid input: {e}")

def generate_nyquist_and_poles(b, c):
    """
    Generate Nyquist plot for L(s) and display poles of closed-loop transfer function T(s).
    """
    with output:
        clear_output(wait=True)
        try:
            # Parse numerator and denominator coefficients
            num = parse_coefficients(b)
            den = parse_coefficients(c)
            
            # Create open-loop transfer function L(s)
            L = control.TransferFunction(num, den)
            
            # Generate Nyquist plot
            plt.figure(figsize=(8,6))
            control.nyquist_plot(L)
            # plt.title('Nyquist Plot of L(s)')
            plt.show()
            
            # Compute closed-loop transfer function T(s) = L(s) / (1 + L(s))
            T = control.feedback(L, 1)
            
            # Get poles of closed-loop transfer function
            poles = control.pole(T)
            
            # # Display closed-loop poles
            # display(Markdown("**Closed-Loop Poles (\( T(s) \)):**"))
            # for idx, pole in enumerate(poles, 1):
            #     display(Markdown(f"- Pole {idx}: {pole:.4f}"))
            
        except ValueError as ve:
            display(Markdown(f"**Error:** {ve}"))
        except Exception as e:
            display(Markdown(f"**An unexpected error occurred:** {e}"))

# Input widgets for numerator and denominator coefficients
numerator_input = widgets.Text(
    value='',
    placeholder='Enter a list of coefficients separated by commas, highest order first',
    description='Numerator:',
    disabled=False
)

denominator_input = widgets.Text(
    value='',
    placeholder='Enter a list of coefficients separated by commas, highest order first',
    description='Denominator:',
    disabled=False
)

# Button to generate Nyquist plot and poles
generate_button = widgets.Button(
    description='Generate Nyquist Plot',
    button_style='success',
    tooltip='Click to generate Nyquist plot and compute closed-loop poles',
    # icon='check'
)

# Output area
output = widgets.Output()

# Event handler for button click
def on_button_click(b):
    generate_nyquist_and_poles(numerator_input.value, denominator_input.value)

generate_button.on_click(on_button_click)

# Display the widgets
display(numerator_input, denominator_input, generate_button, output)



Text(value='', description='Numerator:', placeholder='Enter a list of coefficients separated by commas, highes…

Text(value='', description='Denominator:', placeholder='Enter a list of coefficients separated by commas, high…

Button(button_style='success', description='Generate Nyquist Plot', style=ButtonStyle(), tooltip='Click to gen…

Output()